In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    LlamaForCausalLM, 
    LlamaTokenizer, 
    TrainerCallback, 
    default_data_collator,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

/home/grace-nyutu/Documents/llm-amharic/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype):

    bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    )

    return bnb_config

In [34]:
def load_model(bnb_config):

    model_name = "../model/base/Llama-2-7b-hf"
    n_gpus = torch.cuda.device_count()
    max_memory = f'{21960}MB'
    tokenizer = LlamaTokenizer.from_pretrained(model_name)
    model = LlamaForCausalLM.from_pretrained(
        model_name,
        quantization_config = bnb_config,
        device_map = "auto", 
        torch_dtype=torch.float16 
    )

    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

In [48]:
load_in_4bit = True

bnb_4bit_use_double_quant = True


bnb_4bit_quant_type = "nf4"


bnb_4bit_compute_dtype = torch.bfloat16

bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype)

model, tokenizer = load_model(bnb_config)

RuntimeError: No GPU found. A GPU is needed for quantization.

In [35]:
# Load dataset
dataset_name = "../data/summary.csv"
dataset = load_dataset("csv", data_files = dataset_name, split = "train")

In [36]:
print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

Number of prompts: 95141
Column names are: ['text', 'summary']


In [37]:
from random import randrange

dataset[randrange(len(dataset))]

{'text': '(ሲ.ኤን.ኤን.) - የ14 ዓመቷ Snigdha Nandipati ሐሙስ ምሽት የስክሪፕስ ናሽናል ሆሄሊንግ ንብን "ጉቴፔንስ" ብሎ በመፃፍ አሸንፏል፣ ትርጉሙም አድብቶ፣ ወጥመድ ወይም ወጥመድ ማለት ነው። የሳንዲያጎ የስምንተኛ ክፍል ተማሪ አርብ ጥዋት ለ CNN እንደተናገረው "በአንድ ጊዜ አንድ ቃል እየወሰድኩ ነበር." "እኔ ብቻ እያንዳንዱን ቃል በትክክል ማግኘት ፈልጎ ነበር, እኔ በእርግጥ ለማሸነፍ አላሰብኩም ነበር, በእርግጥ." ከፈረንሳይኛ የተወሰደውን የአሸናፊውን ቃል በትክክል መፃፍ አስቸጋሪ እንዳልሆነ ተናግራለች። ቃሉን ከዚህ በፊት አይታ ነበር እና አውቀዋለች አለች ። ኮንፈቲ መውደቅ እስኪጀምር ድረስ ናንዲፓቲ ድሏን በትክክል አላስመዘገበችም አለች ። "አሸንፋለሁ ብዬ አልጠበኩም ነበር በዚህ አመት ጥሩ ጥሩ ተወዳዳሪዎች ነበሩ" ትላለች። ባለፈው አመት የፊደል አጻጻፍ ንብ 27ኛ ደረጃን አግኝታለች። የብሔራዊ ፊደል ንብ ሥቃይ እና ደስታ። በ13ኛው ዙር የናንዲፓቲ ድል የተገኘው የ14 ዓመቷ የኦርላንዶ የመጨረሻ ተፎካካሪዋ ስቱቲ ሚሽራ “schwarmerei” በሚለው የፊደል አጻጻፍ ከተደናቀፈች በኋላ ነው፤ ይህ ማለት ከልክ ያለፈ፣ ገደብ የለሽ ጉጉት ወይም ትስስር ማለት ነው። የ12 አመቱ የኒውዮርክ ከተማ የሰባተኛ ክፍል ተማሪ የሆነው አርቪንድ ማሃንካሊ "schwannoma" የተባለውን የፔሪፈራል ነርቭ ሽፋን እጢ የተሳሳተ ፊደል ሲጽፍ ሶስተኛ ወጥቷል። @ScrippsBee በትዊተር በላከው መሰረት ናንዲፓቲ በቀን ስድስት ሰአት አጥንቷል። ለማየት ከህንድ በሄዱት ወንድሟ፣ ወላጆቿ (አባቷ አሰለጠኗት) እና አያቶቿ ደስ አሰኘች። ውድድሩ የተካሄደው ከዋሽንግተን ውጭ በሚገኘው በናሽናል ሃርቦር ሜሪላንድ በሚገኘው ጌይሎርድ ናሽናል ሪዞርት እና ኮንቬንሽን

In [38]:
def create_prompt_formats(sample):
    """
    Creates a formatted prompt template for a prompt in the dataset
    """

    # Initialize static strings for the prompt template
    INTRO_BLURB = "Below is an instruction that is a summary of a text. Write a response that appropriately gives the complete text."
    INSTRUCTION_KEY = "### Instruction:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    # Combine a prompt with the static strings
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['summary']}"
    response = f"{RESPONSE_KEY}\n{sample['text']}"
    end = f"{END_KEY}"

    # Create a list of prompt template elements
    parts = [part for part in [blurb, instruction, response, end] if part]

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)

    # Store the formatted prompt template in a new key "text"
    sample["intro"] = formatted_prompt

    return sample

In [39]:
create_prompt_formats(dataset[randrange(len(dataset))])

{'text': 'የአርታዒ ማስታወሻ፡ የMyGoodDeed ተባባሪ መስራች ጄይ ኤስ ዊኑክ በሴፕቴምበር 11 ላይ የአለም ንግድ ማእከል ሳውዝ ታወር ሲወድም በግሌን ጄ. , 2001. በዚህ ሳምንት ግሌን ከዩናይትድ ስቴትስ ኦፍ አሜሪካ በ 9/11 የጀግኖች የቫሎር ሜዳሊያ ተሸልሟል። ጄይ ዊኑክ ሴፕቴምበር 11 በተሻለ ሁኔታ የሚከበረው ለሌሎች አገልግሎት ቀን ነው። ኒው ዮርክ (ሲ.ኤን.) - በመጪው መስከረም 11 ላይ የተፈጸሙት ጥቃቶች ስምንተኛ አመት በሚሊዮኖች ለሚቆጠሩ አሜሪካውያን አሳማኝ ጥያቄ አስነስቷል-ይህንን ልዩ የሆነ አሳዛኝ ቀን በሀገራችን ታሪክ እንዴት በተሻለ ሁኔታ ማክበር አለብን? በእርግጠኝነት, የበዓል ቀን መሆን የለበትም. ይህ ከስራ ለቀናት እረፍት እና የሶስት ቀን ቅዳሜና እሁድ በባርቤኪው እና በነጭ ሽያጭ ለመደሰት ጊዜ አይደለም። አይደለም መስከረም 11 የማሰላሰል ቀን ነውና ታሪካዊ እና ስሜታዊ ትርጉሙ ከጊዜ ወደ ጊዜ እየቀነሰ ወይም በምንም መልኩ መቀነስ የለበትም። በተማሩት ጠቃሚ ትምህርቶች ላይ የሚያተኩርበት ቀን ነው። የ9/11 ቤተሰብ አባል ነኝ። ደፋር ወንድሜ ግሌን ጄ ዊኑክ ከአለም ንግድ ማእከል ሁለት ብሎኮች በሚገኘው ሆላንድ እና ናይት በተባለ ትልቅ የህግ ድርጅት አጋር ነበር። ለ20 ዓመታት ያህል ግሌን የበጎ ፈቃደኝነት የእሳት አደጋ መከላከያ ሠራተኛ እና የድንገተኛ ሕክምና ቴክኒሻን ነበር። ማማዎቹ በተመታ ጊዜ ግሌን የማዳኛ መሳሪያዎችን ተበድሮ ከደህንነት ወደ ደቡብ ታወር በመሮጥ በነፍስ አድን ጥረት ላይ ለመሳተፍ ሄደ። ደቡብ ግንብ ሲፈርስ ሞተ። ከጥቃቶቹ በኋላ ብዙም ሳይቆይ፣ ከጓደኛዬ ዴቪድ ፔይን እና ከሌሎች የ9/11 ቤተሰብ አባላት እና ጓደኞች ጋር፣ MyGoodDeed የሚባል መሰረታዊ ተነሳሽነት ፈጠርኩ። ግባችን? መስከረም 11

In [40]:
def get_max_length(model):
    """
    Extracts maximum token length from the model configuration
    """

    # Pull model configuration
    conf = model.config
    # Initialize a "max_length" variable to store maximum sequence length as null
    max_length = None
    # Find maximum sequence length in the model configuration and save it in "max_length" if found
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    # Set "max_length" to 1024 (default value) if maximum sequence length is not found in the model configuration
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


In [41]:
def preprocess_batch(batch, tokenizer, max_length):
   
    return tokenizer(
        batch["text"],
        max_length = max_length,
        truncation = True,
    )

In [42]:
def preprocess_dataset(tokenizer, max_length, seed, dataset):
  
    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    # Apply preprocessing to each batch of the dataset & and remove "summary", and "text" fields
    preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = ["summary", "text"],
    )

    # Filter out samples that have "input_ids" exceeding "max_length"
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed = seed)

    return dataset

In [49]:
# Random seed
seed = 22

max_length = get_max_length(model)
preprocessed_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset)

NameError: name 'model' is not defined

In [50]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

NameError: name 'lora_alpha' is not defined